In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: shawn
"""
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

## Generate 25 Size and BM double-sorted portfolios

In [2]:
df = pd.read_csv('/Users/shawn/Github/M1/金融計量/TWstocks.csv')
df.head()

,證券代碼,年月,收盤價(元)_月,報酬率％_月,流通在外股數(千股),市值(百萬元),市值比重％,開盤價(元)_月,最高價(元)_月,最低價(元)_月,...,本益比-TEJ,股價淨值比-TSE,股價淨值比-TEJ,股價營收比-TEJ,股利殖利率-TSE,現金股利率,股價漲跌(元),高低價差%,市場別,該月結束日
0,1101 台泥,201001,10.92,-12.2058,3292175,98271,0.516,12.43,13.31,10.68,...,16.14,1.28,1.28,4.57,4.42,4.42,-4.15,21.1765,TSE,20100129
1,1102 亞泥,201001,12.76,-12.4276,2985735,90468,0.475,14.57,15.20,12.57,...,12.67,1.26,1.26,8.56,6.93,5.77,-4.30,18.0636,TSE,20100129
2,1103 嘉泥,201001,9.23,-12.7072,717877,11342,0.060,10.63,11.04,9.06,...,56.97,0.68,0.68,4.50,0.00,0.00,-2.30,18.7845,TSE,20100129
3,1104 環泥,201001,7.54,-7.1428,603891,9813,0.052,8.12,9.10,7.31,...,65.70,0.81,0.81,3.16,0.00,0.00,-1.25,22.0000,TSE,20100129
4,1108 幸福,201001,5.03,-11.5084,404738,3206,0.017,5.68,5.94,4.96,...,NaN,0.75,0.75,1.09,0.00,0.00,-1.03,17.3184,TSE,20100129


## Size

In [3]:
df['帳面市值比'] = 1/df['股價淨值比-TEJ']
# ME = df.groupby('年月')['市值(百萬元)'].apply(lambda x: x.median())
# ME.name = '市值_中位數'
# df = df.merge(ME, on='年月')
# df['市值matrix'] = np.where(df['市值(百萬元)']>df['市值_中位數'], 'B', 'S')
df['市值分組'] = df.groupby('年月')['市值(百萬元)'].transform(lambda x: pd.qcut(x, q=5, labels=['ME1', 'ME2', 'ME3', 'ME4', 'ME5']))

In [4]:
df['帳面市值比分組'] = df.groupby('年月')['帳面市值比'].transform(lambda x: pd.qcut(x, q=5, labels=['BM1', 'BM2', 'BM3', 'BM4', 'BM5']))

In [5]:
df

,證券代碼,年月,收盤價(元)_月,報酬率％_月,流通在外股數(千股),市值(百萬元),市值比重％,開盤價(元)_月,最高價(元)_月,最低價(元)_月,...,股價營收比-TEJ,股利殖利率-TSE,現金股利率,股價漲跌(元),高低價差%,市場別,該月結束日,帳面市值比,市值分組,帳面市值比分組
0,1101 台泥,201001,10.92,-12.2058,3292175,98271,0.516,12.43,13.31,10.68,...,4.57,4.42,4.42,-4.15,21.1765,TSE,20100129,0.781250,ME5,BM4
1,1102 亞泥,201001,12.76,-12.4276,2985735,90468,0.475,14.57,15.20,12.57,...,8.56,6.93,5.77,-4.30,18.0636,TSE,20100129,0.793651,ME5,BM4
2,1103 嘉泥,201001,9.23,-12.7072,717877,11342,0.060,10.63,11.04,9.06,...,4.50,0.00,0.00,-2.30,18.7845,TSE,20100129,1.470588,ME4,BM5
3,1104 環泥,201001,7.54,-7.1428,603891,9813,0.052,8.12,9.10,7.31,...,3.16,0.00,0.00,-1.25,22.0000,TSE,20100129,1.234568,ME4,BM5
4,1108 幸福,201001,5.03,-11.5084,404738,3206,0.017,5.68,5.94,4.96,...,1.09,0.00,0.00,-1.03,17.3184,TSE,20100129,1.333333,ME2,BM5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141420,9944 新麗,202312,20.20,-1.2224,109107,2204,0.004,20.45,20.90,20.05,...,1.00,5.20,5.20,-0.25,4.1565,TSE,20231226,1.612903,ME1,BM5
141421,9945 潤泰新,202312,36.70,2.5140,2844225,104383,0.186,35.80,37.70,35.70,...,3.58,0.00,0.00,0.90,5.5866,TSE,20231226,0.813008,ME5,BM4
141422,9946 三發地產,202312,20.50,0.0000,326554,6694,0.012,20.50,21.65,18.95,...,7.81,2.10,2.09,0.00,13.1707,TSE,20231226,0.952381,ME3,BM4
141423,9955 佳龍,202312,24.70,-0.8032,103208,2549,0.005,25.00,27.10,23.75,...,2.27,0.00,0.00,-0.20,13.4538,TSE,20231226,0.526316,ME1,BM3


In [6]:
df['size_bm'] = np.where((df['市值分組'] == 'ME1') & (df['帳面市值比分組'] == 'BM1'), 'ME1_BM1',
                np.where((df['市值分組'] == 'ME1') & (df['帳面市值比分組'] == 'BM2'), 'ME1_BM2',
                np.where((df['市值分組'] == 'ME1') & (df['帳面市值比分組'] == 'BM3'), 'ME1_BM3',
                np.where((df['市值分組'] == 'ME1') & (df['帳面市值比分組'] == 'BM4'), 'ME1_BM4',
                np.where((df['市值分組'] == 'ME1') & (df['帳面市值比分組'] == 'BM5'), 'ME1_BM5',
                np.where((df['市值分組'] == 'ME2') & (df['帳面市值比分組'] == 'BM1'), 'ME2_BM1',
                np.where((df['市值分組'] == 'ME2') & (df['帳面市值比分組'] == 'BM2'), 'ME2_BM2',
                np.where((df['市值分組'] == 'ME2') & (df['帳面市值比分組'] == 'BM3'), 'ME2_BM3',
                np.where((df['市值分組'] == 'ME2') & (df['帳面市值比分組'] == 'BM4'), 'ME2_BM4',
                np.where((df['市值分組'] == 'ME2') & (df['帳面市值比分組'] == 'BM5'), 'ME2_BM5',
                np.where((df['市值分組'] == 'ME3') & (df['帳面市值比分組'] == 'BM1'), 'ME3_BM1',
                np.where((df['市值分組'] == 'ME3') & (df['帳面市值比分組'] == 'BM2'), 'ME3_BM2',
                np.where((df['市值分組'] == 'ME3') & (df['帳面市值比分組'] == 'BM3'), 'ME3_BM3',
                np.where((df['市值分組'] == 'ME3') & (df['帳面市值比分組'] == 'BM4'), 'ME3_BM4',
                np.where((df['市值分組'] == 'ME3') & (df['帳面市值比分組'] == 'BM5'), 'ME3_BM5',
                np.where((df['市值分組'] == 'ME4') & (df['帳面市值比分組'] == 'BM1'), 'ME4_BM1',
                np.where((df['市值分組'] == 'ME4') & (df['帳面市值比分組'] == 'BM2'), 'ME4_BM2',
                np.where((df['市值分組'] == 'ME4') & (df['帳面市值比分組'] == 'BM3'), 'ME4_BM3',
                np.where((df['市值分組'] == 'ME4') & (df['帳面市值比分組'] == 'BM4'), 'ME4_BM4',
                np.where((df['市值分組'] == 'ME4') & (df['帳面市值比分組'] == 'BM5'), 'ME4_BM5',
                np.where((df['市值分組'] == 'ME5') & (df['帳面市值比分組'] == 'BM1'), 'ME5_BM1',
                np.where((df['市值分組'] == 'ME5') & (df['帳面市值比分組'] == 'BM2'), 'ME5_BM2',
                np.where((df['市值分組'] == 'ME5') & (df['帳面市值比分組'] == 'BM3'), 'ME5_BM3',
                np.where((df['市值分組'] == 'ME5') & (df['帳面市值比分組'] == 'BM4'), 'ME5_BM4',
                np.where((df['市值分組'] == 'ME5') & (df['帳面市值比分組'] == 'BM5'), 'ME5_BM5',
                         ''))))))))))))))))))))))))) # 如果沒有符合的條件，給予 NaN

In [7]:
df.drop(df[df['size_bm'] == ''].index, inplace=True)
df.head()


,證券代碼,年月,收盤價(元)_月,報酬率％_月,流通在外股數(千股),市值(百萬元),市值比重％,開盤價(元)_月,最高價(元)_月,最低價(元)_月,...,股利殖利率-TSE,現金股利率,股價漲跌(元),高低價差%,市場別,該月結束日,帳面市值比,市值分組,帳面市值比分組,size_bm
0,1101 台泥,201001,10.92,-12.2058,3292175,98271,0.516,12.43,13.31,10.68,...,4.42,4.42,-4.15,21.1765,TSE,20100129,0.781250,ME5,BM4,ME5_BM4
1,1102 亞泥,201001,12.76,-12.4276,2985735,90468,0.475,14.57,15.20,12.57,...,6.93,5.77,-4.30,18.0636,TSE,20100129,0.793651,ME5,BM4,ME5_BM4
2,1103 嘉泥,201001,9.23,-12.7072,717877,11342,0.060,10.63,11.04,9.06,...,0.00,0.00,-2.30,18.7845,TSE,20100129,1.470588,ME4,BM5,ME4_BM5
3,1104 環泥,201001,7.54,-7.1428,603891,9813,0.052,8.12,9.10,7.31,...,0.00,0.00,-1.25,22.0000,TSE,20100129,1.234568,ME4,BM5,ME4_BM5
4,1108 幸福,201001,5.03,-11.5084,404738,3206,0.017,5.68,5.94,4.96,...,0.00,0.00,-1.03,17.3184,TSE,20100129,1.333333,ME2,BM5,ME2_BM5


In [8]:
df.to_csv('/Users/shawn/Github/M1/金融計量/test.csv')

In [9]:
weighted_returns = df.groupby(['年月', 'size_bm']).apply(lambda x: (x['報酬率％_月'] * x['市值(百萬元)']).sum() / x['市值(百萬元)'].sum()).reset_index()
weighted_returns.columns = ['date', 'portf', 'ExRet']

In [10]:
weighted_returns

,date,portf,ExRet
0,201001,ME1_BM1,32.890875
1,201001,ME1_BM2,-4.695570
2,201001,ME1_BM3,-8.750316
3,201001,ME1_BM4,-8.279946
4,201001,ME1_BM5,-5.332219
...,...,...,...
4195,202312,ME5_BM1,2.298384
4196,202312,ME5_BM2,3.594205
4197,202312,ME5_BM3,2.144609
4198,202312,ME5_BM4,1.200289


In [14]:
result_df = weighted_returns.pivot(index='date', columns='portf', values='ExRet')
result_df

portf,ME1_BM1,ME1_BM2,ME1_BM3,ME1_BM4,ME1_BM5,ME2_BM1,ME2_BM2,ME2_BM3,ME2_BM4,ME2_BM5,...,ME4_BM1,ME4_BM2,ME4_BM3,ME4_BM4,ME4_BM5,ME5_BM1,ME5_BM2,ME5_BM3,ME5_BM4,ME5_BM5
date,,,,,,,,,,,,,,,,,,,,,
201001,32.890875,-4.695570,-8.750316,-8.279946,-5.332219,-3.561119,-6.206268,-8.363253,-6.089003,-6.524916,...,-9.094658,-7.414896,-5.943013,-5.977922,-8.179228,-6.186834,-2.092595,-5.602656,-7.959870,-8.790981
201002,0.461274,-6.456261,-0.365751,-3.706888,-1.921799,4.965301,0.281530,-0.301325,-2.389591,-4.198519,...,-1.012413,-2.748699,-4.571871,-3.777080,-2.495989,-2.822378,1.146516,-4.685273,-3.489328,-3.885512
201003,8.010449,15.595326,16.829004,11.374620,9.151004,17.013821,13.654096,14.652281,10.992721,8.537164,...,13.410443,11.032410,8.539586,8.267648,8.199345,7.165132,1.894917,5.862600,5.690239,8.779601
201004,9.335599,-1.397550,6.335519,4.250821,4.553978,3.237774,8.009871,0.927501,1.876041,0.091638,...,1.882787,2.095164,2.166175,-1.823111,0.449869,2.688093,0.879703,2.068281,-0.033417,-0.401744
201005,-10.396295,-2.172570,-8.820830,-8.777200,-7.532012,-6.222561,-7.443623,-11.024121,-8.652532,-10.632070,...,-7.716726,-6.881043,-9.721227,-10.950684,-8.869110,-5.924697,-7.187865,-6.917847,-9.152587,-7.581057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202308,14.557165,-0.850011,-0.156880,-1.426551,-3.604449,0.736321,-3.556921,-6.560576,-1.825461,-3.469404,...,-1.729865,-0.491141,2.001804,2.071255,-2.987260,-1.963964,-1.894873,-2.223568,-2.764339,-3.847808
202309,5.051835,-0.836928,0.763077,-0.893305,0.217345,-2.863591,1.759109,1.297324,1.355087,-0.574739,...,2.708116,1.199242,0.234765,-1.074088,-1.053402,-2.677368,0.082486,1.644722,-1.052130,0.063900
202310,-2.909976,-5.554652,-1.611277,-2.235559,-2.517852,-1.189402,-1.284320,1.866707,-3.311658,-0.983407,...,-4.408733,-2.542426,-3.232488,-2.321728,-2.953364,-0.503581,-3.152601,-1.717375,-2.340082,-4.494596


In [15]:
result_df.to_csv('/Users/shawn/Github/M1/金融計量/25_Size_BM.csv')